# PRACTICAL Lesson  5 

##MARKOV CHAINS


## CONTENT

5.1. INTRODUCTION

5.2. PRACTICAL EXAMPLE. HOW A STOCK SHARE EVOLVES

5.3. PROPOSED EXERCISE: WILL THE RICH BECOME RICHER AND THE POOR BECOME POORER?

5.4. PROPOSED PROBLEM. SEPTEMBER 2002 EXAMINATION

5.5. PROPOSED EXERCISES. REPAIRING RIVETING MACHINES


In [1]:
!pip install networkx
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install ipywidgets


### 5.1 INTRODUCTION

In decision-making problems, the need to make decisions based on phenomena with associated uncertainty frequently arises.

A stochastic process is defined as a series of aleatory variables $\{\mathrm{Xt}\}$, where index $t$ takes the values of a given set $T$. For example, process $X 1, X 2, X 3, \ldots$ can represent a change made by customers in a detergent brand name.

Another simple case is that in which knowledge about the state of the system in consecutive instants prior to $t$ provides certain information on the state of the system, although this information is totally contained in the last instant. A process with these characteristics is called a Markov Process. For instance, if the change in a detergent brand name made by customers in one month depends only on their choice in the previous month.

A stochastic process $\{\mathrm{Xt}\}$ is a Markov Chain if it has the following characteristics:

1. A finite number of states.
2. The Markovian property. This property is the equivalent to establishing that the conditional probability of any future "event", given any past "event" and the present state, is independent of the past event and that it depends only on the current process state.
3. Stationary transition probabilities. Having stationary transition probabilities means that the transition probabilities do not change over time.
4. A series of initial probabilities.

If the transition probabilities (in the probabilities example of customers changing from one detergent brand name (state) to another) continue to be constant with time, the Markov Process is called a Markov Chain.

An example of a Markov Chain: let's consider the following model for the a share value. At the end of a given day, the price is recorded. If the share goes up, the probability of it going up tomorrow is 0.7 . If the share goes down, the probability of it going up tomorrow is only 0.5 . This is a Markov Chain where state $0(\mathrm{e} 0)$ represents the share price going up and state 1 (e1) represents it going down. The transition Matrix is given by:

| States | E0=S | E1=B |
| :---: | :---: | :---: |
| E0=S | 0.70 | 0.30 |
| E1=B | 0.50 | 0.50 |

In [4]:
execfile("vis.py")


In [5]:
m = [[ 0.9, 0.1, 0.0, 0.0],
     [ 0.0, 0.0, 0.5 ,0.5],
     [0.6, 0.0, 0.0 ,0.4],
     [0.0, 0.0, 0.3, 0.7]]
np.linalg.matrix_power(m, 2)[0,1]-np.linalg.matrix_power(m, 1)[0,1]*0.1

0.08000000000000002

In [7]:
np.linalg.matrix_power(m, 2)

array([[0.81, 0.09, 0.05, 0.05],
       [0.3 , 0.  , 0.15, 0.55],
       [0.54, 0.06, 0.12, 0.28],
       [0.18, 0.  , 0.21, 0.61]])

In [14]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import display
import base64

def render_latex_to_image_base64(latex_str):
    """
    Renders LaTeX string to an image and returns a base64 encoded string.

    Args:
    latex_str (str): LaTeX string to render.

    Returns:
    str: Base64 encoded string of the rendered image.
    """
    # Create a figure for rendering
    fig = plt.figure()
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)

    # Use the LaTeX string as text in the plot
    ax.text(0.5, 0.5, f"${latex_str}$", fontsize=20, va='center', ha='center', color='black')

    # Hide the axes
    ax.axis('off')
    fig.patch.set_visible(False)
    ax.patch.set_visible(False)

    # Render the figure to a PNG
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)

    # Encode the image to base64
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    
    return img_base64
def create_latex_button(img_base64, button_width, button_height):
    # Create an Image widget to display the rendered LaTeX
    image_widget = widgets.Image(
        value=img_base64,
        format='png',
        width=button_width,
        height=button_height
    )

    # Create a transparent Button widget
    button = widgets.Button(
        layout=widgets.Layout(width='100%', height='100%', opacity=0)
    )
    button.style.button_color = 'transparent'

    # Function to handle button clicks
    def on_button_click(b):
        print("Button clicked!")

    button.on_click(on_button_click)

    # Overlay the Button on the Image using VBox
    overlay = widgets.VBox([image_widget, button])
    overlay.layout = widgets.Layout(
        width=f'{button_width}px', 
        height=f'{button_height}px', 
        align_items='center', 
        justify_content='center'
    )

    return overlay

# Create the LaTeX button widget
latex_button = create_latex_button(img_base64, 200, 100)
display(latex_button)

TraitError: The 'value' trait of an Image instance expected a memory view object, not the str 'iVBORw0KGgoAAAANSUhEUgAAAoAAAAHgCAYAAAA10dzkAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8pXeV/AAAACXBIWXMAAA9hAAAPYQGoP6dpAAAL6UlEQVR4nO3dO6hlZxmA4e+fRBKYVixUAooJliKKhResAokWYqHxRiam0FQaEC0sjATERoldIIlEIsQLgVh4RcFbKUFELQQFtdGIKKhFEs1vcbYwDDNkYo7nmHmfp9l7rf1vzle+Z+11WXvvAQCg48xpDwAAwMkSgAAAMQIQACBGAAIAxAhAAIAYAQgAECMAAQBiBCAAQIwABACIEYAAADECEAAgRgACAMQIQACAGAEIABAjAAEAYgQgAECMAAQAiBGAAAAxAhAAIEYAAgDECEAAgBgBCAAQIwABAGIEIABAjAAEAIgRgAAAMQIQACBGAAIAxAhAAIAYAQgAECMAAQBiBCAAQIwABACIEYAAADECEAAgRgACAMQIQACAGAEIABAjAAEAYgQgAECMAAQAiBGAAAAxAhAAIEYAAgDECEAAgBgBCAAQIwABAGIEIABAjAAEAIgRgAAAMQIQACBGAAIAxAhAAIAYAQgAECMAAQBiBCAAQIwABACIEYAAADECEAAgRgACAMQIQACAGAEIABAjAAEAYgQgAECMAAQAiBGAAAAxAhAAIEYAAgDECEAAgBgBCAAQIwABAGIEIABAjAAEAIgRgAAAMQIQACBGAAIAxAhAAIAYAQgAECMAAQBiBCAAQIwABACIEYAAADECEAAgRgACAMQIQACAGAEIABAjAAEAYgQgAECMAAQAiBGAAAAxAhAAIEYAAgDECEAAgBgBCAAQIwABA


### 5.2 PRACTICAL EXAMPLE: HOW A STOCK SHARE EVOLVES

A Stock Market analyst has observed the evolution of an electronic products firm's shares over the last few months, and concludes that this evolution can be modelled with a Markov Chain.

The probability that the share goes up or down on a given day depends on what has happened on the 2 previous days:

Indeed, if the share went up on 2 days (today and yesterday), the probability of it going up tomorrow is 0.9 . If it went up today but went down yesterday, the probability of it going up tomorrow is 0.6 . If it went down today but went up yesterday, the probability of it going up tomorrow is 0.5 . Finally, if it went down on the last 2 days, the probability of it going up tomorrow is 0.3 .

## QUESTIONS:


### What are the Markov Chain states?


### What is the one-step transition probabilities matrix?


(The next questions must be solved with the help of the CaDMGip software)

- What is the probability of a share going up or down on a given day?
- What is the probability of a share that went up yesterday and today going down the day after tomorrow?
- The transition probabilities matrix with 2, 3,10 and 30 steps
- The first-passage times matrix.




### 5.3 PROPOSED EXERCISE: WILL THE RICH BECOME RICHER, AND THE POOR BECOME POORER?

During the 1992 American presidential campaign, the Democrat candidate, Bill Clinton, defending he would raise taxes for the "rich" and would give some relief to tax payers earning average and low salaries.

One of the US Treasury's functions is to study tax proposals and to determine how they could affect the country (the economy, distribution of the population's income, and other concepts). One of the studies conducted by this Department, based on 14,351 tax payers and tax declarations between 1979 and 1988, discovered that a
considerable number of Americans on low income would move up on the income scale, while those who were higher up on the income scale would stay where they were.

The data in Table 1 are the percentages of tax payers who moved between the various levels of income at the end of the decade (adjusted by inflation). For instance, $47.3 \%$ of tax payers who earned $\$ 200,000$ or more at the beginning of the decade remained in this category at the end of this period; the income of $38.6 \%$ of tax payers went down to the next category (between $\$ 45,000$ and $\$ 199,000$ ); the income of $7.7 \%$ went down and fell within the $\$ 25,000-44,900$ range, and even $2.2 \%$ earned the income of "poor people, that is, less than $\$ 7,000$.

| Table 1. Shift in Income |  |  |  |  |  |  |  |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
|  | Income at the end of the decade |  |  |  |  |  |  |
| Income at the <br> beginning of <br> the decade | $\$ 200,000$ <br> or more | $\$ 45,000$ <br> to <br> $\$ 199,999$ | $\$ 25,000$ to <br> $\$ 44,999$ | $\$ 15,000$ to <br> $\$ 24,999$ | $\$ 7,000$ to <br> $\$ 14,999$ | Less than <br> $\$ 7,000$ |  |
| $\$ 200,000$ or <br> more | $47.3 \%$ | $38.6 \%$ | $7.7 \%$ | $3.8 \%$ | $0.4 \%$ | $2.2 \%$ |  |
| $\$ 45,000$ to <br> $\$ 199,999$ | 5.3 | 59.4 | 20.3 | 9.4 | 4.4 | 1.2 |  |
| $\$ 25,000$ to <br> $\$ 44,999$ | 0.6 | 34.8 | 37.5 | 14.8 | 9.3 | 3.0 |  |
| $\$ 15,000$ to <br> $\$ 24,999$ | 0.4 | 14.6 | 32.3 | 33.0 | 14.0 | 5.7 |  |
| $\$ 7,000$ to <br> $\$ 14,999$ | 0.3 | 10.8 | 19.5 | 29.6 | 29.0 | 10.8 |  |
| Less than <br> $\$ 7,000$ | 0.3 | 14.4 | 25.3 | 25.0 | 20.7 | 14.3 |  |

Sarah $\mathrm{Hu}$, a senior analyst in a non-profit institute, has been assigned a project to analyse the distribution of the US population's income, and also the effect that the various proposed taxes might have. Recently, some groups have shown an interest in stressing that in the USA, "the rich are becoming richer and the poor are becoming poorer". Ms. Hu has been commissioned to determine whether she can shed some light on this matter with the results of the US Treasury Study. She feels particularly intrigued by the similarity between Table 1 and some Markov Processes that she studied when she was at University.

Ms. Hu has estimated that, currently, $1 \%$ of the US population earns more than $\$ 200,000$; $19 \%$ earn between $\$ 45,000$ and $\$ 199,000$; the income of $20 \%$ falls between $\$ 25,000$ and $\$ 44,999$; the earnings of $20 \%$ fall within the $\$ 15,000-24,999$ range; another $20 \%$ earn somewhere between $\$ 7,000$ and $\$ 14,999$; finally, the income of $20 \%$ is below $\$ 7,000$.

## QUESTIONS:

1. What are the Markov Chain states?
2. Based on the data in Table 1, what will the long-term income distribution be?
3. What percentage of the population who earn less than $\$ 7,000$ will earn more than $\$ 200,000$ after 20 years? What is the percentage for $\$ 100,000$ ?
4. What is the expected number of years that the tax payers who earn under $\$ 7,000$ will earn more than $\$ 200,000$ ? And $\$ 100,000$ ?
5. How has the income distribution changed in relation to the initial income distribution?
6. Using the CADMGIP tool to help you, respond to the question that the problem considers; will the rich become richer, and the poor become poorer?



### 5.4 PROPOSED PROBLEM.

At the beginning of each year, my car can be in a good, regular or bad state. There is a $90 \%$ probability of a good car still being good at the beginning of the next year, a $5 \%$ one of it being in a regular state, and a $5 \%$ one of it being in a bad state. There is s 70\% probability that a car in a regular state will still be regular at the beginning of the next year, and a $30 \%$ one that it will be in a bad state. It costs 12,000 to buy a good car, 5,000 euros to buy a regular car (5000 euros is also the price to sell it), but a bad car has no sale value and must be immediately replaced with a good car. The cost to maintain a car in a good state is 1,000 euros a year, and 2,000 euros a year for a car in a regular state. Let's assume that the cost to run a car per year depends on the type of car the driver has at the beginning of the year..

Let's assume that a car in a bad state only happens at the end of a year, and then (at the beginning of the next year) the car in a bad state "must be replaced immediately". Should I replace my car as soon as it is in a regular state, or should I wait until it is in a bad state? To solve this matter:

a) Define the Markov chains for the three states (Good, Regular and Bad at the beginning of the year) for all the replacement policies (Policy A: Replace when the car is in a bad state; Policy B: Replace when the car is in a regular state).

b) Obtain the steady-state probabilities.

c) Determine the mean cost per year for each replacement policy.

d) What replacement policy would you recommend?




### 5.5 PROPOSED EXERCISE. REPAIRING RIVETING MACHINES

On a daily basis, a maintenance technician of a given firm checks adjustments on an automatic machine that produces rivets for the air-sea industry, among other tasks.

By checking a certain number of rivets, he knows that it is not necessary to adjust the machine (A), it requires slight adjustment (B), some adjustment (C) or considerable adjustment (D).

The technician has verified that if the machine is in state A, there is an $80 \%$ probability that it will remain in this state until the next day, and a $20 \%$ probability that it will enter state $B$.

If the machine is in state $B$, there is a $70 \%$ probability that it will remain in this state until the next day, and a $5 \%$ probability that it will require considerable adjustment.

If the machine is in state $\mathrm{C}$, there is a $50 \%$ probability that it will require considerable adjustment, and it is quite certain that its state will not improve.

If the machine requires considerable adjustment, it will remain in this state permanently.

However, the technician's job does not consist in stopping and observing the machine; instead, if the machine is found to be in state $D$, it must be adjusted. Nevertheless, the technician is not clear if when the machine is in any of the other states what interests the firm more from an economic perspective.

This is because a complete adjustment costs 20,000 m.u. If the machine is in state A and begins to work at the beginning of the day, it will not produce any extra cost through loss of quality. If the technician adjusts it in state $B$, the cost for the firm is 5,000 m.u. that day, and it is 8,000 m.u. if the machine is in state C.

The maintenance technician needs to economically evaluate if it is less costly to also adjust the machine completely when it is in state C.

a) Model the problem with Markov Chains (one per maintenance policy) $(40 \%)$

b) Which policy is less costly? (40\%)

c) What would the adjustment cost be so that the answer to question b) would have been the opposite? $(10 \%)$

d) With the original maintenance policy, what is the expected firstpassage time from state $D$ to state $A$ ? What is the recurrence time of state $D$ ? (10\%)
